In [1]:
import os
import torch
import torch.nn as nn
import torchvision.models as models
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
import numpy as np
from tqdm import tqdm

In [2]:
class CustomImageDataset(Dataset):
    def __init__(self, data, labels=None, transform=None):
        self.data = data
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image = self.data[idx]
        label = self.labels[idx] if self.labels is not None else -1
        if self.transform:
            image = self.transform(image)
        return image, label


transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load ResNet-152 and fine-tune last layers
device = 'cuda' if torch.cuda.is_available() else 'cpu'
feature_extractor = models.resnet152(weights=models.ResNet152_Weights.DEFAULT)
feature_extractor = nn.Sequential(*list(feature_extractor.children())[:-1])  # Remove final layer
feature_extractor.eval().to(device)  # Move model to device

Downloading: "https://download.pytorch.org/models/resnet152-f82ba261.pth" to /root/.cache/torch/hub/checkpoints/resnet152-f82ba261.pth
100%|██████████| 230M/230M [00:01<00:00, 154MB/s]


Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)


In [3]:
# Enable fine-tuning of the last layers for Task 2
for param in feature_extractor[-1].parameters():
    param.requires_grad = True

In [4]:
def extract_features(model, dataloader, device='cpu'):
    """Extracts features for images using the feature extractor."""
    features, labels = [], []
    model.to(device)
    with torch.no_grad():
        for images, lbls in dataloader:
            images = images.to(device)
            feats = model(images).squeeze()  # Extract features
            feats[torch.isnan(feats)] = 0  # Replace NaNs in features
            features.append(feats.cpu())  # Store features on CPU to save GPU memory
            labels.append(lbls)
    return torch.cat(features, dim=0), torch.cat(labels, dim=0)

In [5]:
def compute_prototypes(features, labels, num_classes=10):
    """Computes class prototypes as the mean of feature vectors for each class."""
    prototypes = []
    for cls in range(num_classes):
        class_features = features[labels == cls]
        if class_features.size(0) == 0:  # Handle empty classes
            prototype = torch.zeros(features.size(1))  # Use a zero vector
        else:
            prototype = class_features.mean(dim=0)
        prototype[torch.isnan(prototype)] = 0  # Replace NaNs in prototypes
        prototypes.append(prototype)
    return torch.stack(prototypes)

In [6]:
def classify_with_prototypes(prototypes, query_features):
    """Classifies query features by finding the nearest prototype."""
    distances = torch.cdist(query_features, prototypes)  # Pairwise distances
    return distances.argmin(dim=1)  # Nearest prototype index

In [7]:
def fine_tune_feature_extractor(loader, optimizer, loss_fn, feature_extractor, device, epochs=5):
    """Fine-tune the feature extractor for Task 2 datasets."""
    feature_extractor.train()  # Enable training mode
    for epoch in range(epochs):
        total_loss = 0.0
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = feature_extractor(images).squeeze()
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {total_loss:.4f}")

In [10]:
import zipfile

# Path to the zip file
zip_file_path = '/content/eval_data.zip'

# Destination folder to extract the files
extract_to = '/content/eval_data'

# Unzipping the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print(f"Extracted all files to: {extract_to}")

Extracted all files to: /content/eval_data


In [15]:
def train_sequential_models_task2(starting_prototypes):
    """Trains sequential models for Task 2 datasets."""
    model_prototypes = [starting_prototypes]
    accuracies = []

    # Define fine-tuning components
    optimizer = torch.optim.Adam(feature_extractor.parameters(), lr=1e-4)
    loss_fn = nn.CrossEntropyLoss()

    # Process datasets D11 to D20
    for i in tqdm(range(1, 11), desc="Processing Training Datasets (Task 2)"):
        data_path = f'train_data/train_data/{i}_train_data.tar.pth'
        dataset = torch.load(data_path)
        images, labels = dataset['data'], dataset.get('targets', None)

        # Feature extraction
        dataset = CustomImageDataset(images, labels, transform=transform)
        loader = DataLoader(dataset, batch_size=32, shuffle=True)

        # Fine-tune the feature extractor for the current dataset
        if labels is not None:
            fine_tune_feature_extractor(loader, optimizer, loss_fn, feature_extractor, device)

        # Extract features
        features, lbls = extract_features(feature_extractor, loader, device)

        # Normalize features before computing prototypes to avoid numerical instability
        features = features / (features.norm(dim=1, keepdim=True) + 1e-8)

        # Update prototypes using weighted averaging (adaptation for domain shift)
        predicted_labels = classify_with_prototypes(model_prototypes[-1], features)
        new_prototypes = compute_prototypes(features, predicted_labels)

        # Normalize prototypes to avoid numerical instability
        new_prototypes = new_prototypes / (new_prototypes.norm(dim=1, keepdim=True) + 1e-8)

        # Blend new prototypes with the previous prototypes
        alpha = 0.5  # Lower alpha for better stability across distributions
        prototypes = alpha * new_prototypes + (1 - alpha) * model_prototypes[-1]
        model_prototypes.append(prototypes)
        print(f"Prototypes after D{i}: {prototypes}")

        # Evaluate on all held-out datasets up to current model
        model_accuracies = []
        for eval_idx in tqdm(range(1, i + 11), desc=f"Evaluating Model f{i+10}"):
            eval_data = torch.load(f'eval_data/eval_data/{eval_idx}_eval_data.tar.pth')
            eval_images, eval_labels = eval_data['data'], eval_data['targets']
            eval_dataset = CustomImageDataset(eval_images, eval_labels, transform=transform)
            eval_loader = DataLoader(eval_dataset, batch_size=32, shuffle=False)

            eval_features, eval_lbls = extract_features(feature_extractor, eval_loader, device)

            # Normalize evaluation features
            eval_features = eval_features / (eval_features.norm(dim=1, keepdim=True) + 1e-8)

            predicted_labels = classify_with_prototypes(prototypes, eval_features)

            accuracy = (predicted_labels == eval_lbls).float().mean().item()
            model_accuracies.append(accuracy)
            print(f"Model f{i+10} accuracy on D̂{eval_idx}: {accuracy * 100:.2f}%")

        accuracies.append(model_accuracies)

    return accuracies

In [12]:
# Load starting prototypes from Task 1 (f10)
prototype_file_path = "prototypes_f10.pth"

In [13]:
starting_prototypes = torch.load(prototype_file_path)

<ipython-input-13-6f585dc3681a>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  starting_prototypes = torch.load(prototype_file_path)


In [16]:
# Train models for Task 2
accuracy_matrix_task2 = train_sequential_models_task2(starting_prototypes)
print("\nAccuracy Matrix (Task 2):")
for row in accuracy_matrix_task2:
    print(row)

Processing Training Datasets (Task 2):   0%|          | 0/10 [00:00<?, ?it/s]<ipython-input-15-0ebfb200d6ac>:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  dataset = torc

Prototypes after D1: tensor([[0.0569, 0.0677, 0.0230,  ..., 0.1272, 0.0118, 0.0047],
        [0.0060, 0.0096, 0.0518,  ..., 0.1307, 0.0096, 0.0048],
        [0.0331, 0.0511, 0.0186,  ..., 0.0216, 0.0195, 0.0096],
        ...,
        [0.0081, 0.0302, 0.0345,  ..., 0.0128, 0.0058, 0.0052],
        [0.0097, 0.0334, 0.1497,  ..., 0.0593, 0.0051, 0.0045],
        [0.0055, 0.0169, 0.0262,  ..., 0.0489, 0.0144, 0.0037]])



Evaluating Model f11:   0%|          | 0/11 [00:00<?, ?it/s]<ipython-input-15-0ebfb200d6ac>:46: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  eval_data = torch.load(f'eval_

Model f11 accuracy on D̂1: 44.20%



Evaluating Model f11:  18%|█▊        | 2/11 [00:43<03:16, 21.84s/it]

...

Model f20 accuracy on D̂18: 70.44%



Evaluating Model f20:  95%|█████████▌| 19/20 [06:46<00:21, 21.38s/it]

Model f20 accuracy on D̂19: 59.32%



Processing Training Datasets (Task 2): 100%|██████████| 10/10 [58:48<00:00, 352.89s/it]

Model f20 accuracy on D̂20: 80.88%

Accuracy Matrix (Task 2):
[0.44200000166893005, 0.44519999623298645, 0.4399999976158142, 0.4431999921798706, 0.4375999867916107, 0.43959999084472656, 0.4352000057697296, 0.4472000002861023, 0.4323999881744385, 0.44679999351501465, 0.36800000071525574]
[0.5612000226974487, 0.5383999943733215, 0.5568000078201294, 0.5684000253677368, 0.5472000241279602, 0.5583999752998352, 0.5547999739646912, 0.5587999820709229, 0.52920001745224, 0.5595999956130981, 0.4359999895095825, 0.3508000075817108]
[0.7324000000953674, 0.7364000082015991, 0.7408000230789185, 0.7487999796867371, 0.7328000068664551, 0.7504000067710876, 0.7432000041007996, 0.7336000204086304, 0.7271999716758728, 0.7459999918937683, 0.5956000089645386, 0.4643999934196472, 0.6488000154495239]
[0.800000011920929, 0.8080000281333923, 0.7983999848365784, 0.8127999901771545, 0.7972000241279602, 0.8148000240325928, 0.8023999929428101, 0.8023999929428101, 0.795199990272522, 0.8055999875068665, 0.66879999637